In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np

import chess
import chess.pgn
from stockfish import Stockfish
import os

import uuid        # for id generation
import shortuuid

%load_ext autoreload
%autoreload 2

In [3]:
#stockfish = Stockfish(
#    'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#    parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
#)
#stockfish.set_elo_rating(2600)
#stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'FEN_moves': [],
    'Bitmap_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'turn': [],
    'Castling_right': [],
    'EP_option': [],
    'Pseudo_EP_option': [],
    'Halfmove_clock': []
    #'Result': [],
}

# Set list of Pieces
PIECES = [chess.Piece.from_symbol('P'),
         chess.Piece.from_symbol('N'),
         chess.Piece.from_symbol('B'),
         chess.Piece.from_symbol('R'),
         chess.Piece.from_symbol('Q'),
         chess.Piece.from_symbol('K'),
         chess.Piece.from_symbol('p'),
         chess.Piece.from_symbol('n'),
         chess.Piece.from_symbol('b'),
         chess.Piece.from_symbol('r'),
         chess.Piece.from_symbol('q'),
         chess.Piece.from_symbol('k')]

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # MOVE CYCLE
        white = True
        for move in moves:
            board.push(move)
            #fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
            #cpl = stockfish.get_evaluation()['value']/100
            
            moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
            moves_log_dict['FEN_moves'].append(board.fen())
            
            #Generate bitmap representation of FENs
            bitmap_board_dict = {}
            positions = board.piece_map()

            for piece in PIECES:
                bitmap_board = {}
                for position in positions:
                    if positions[position] == piece: 
                        bitmap_board[position] = 1
                    else:
                        bitmap_board[position] = 0
                bitmap_board_dict[str(piece)] = bitmap_board            
            
            moves_log_dict['Bitmap_moves'].append(bitmap_board_dict)
            
            #moves_log_dict['cpl'].append(cpl)
            
            #Turn color and castling availablity
            moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
            if white:
                moves_log_dict['turn'].append('white')
                moves_log_dict['Castling_right'].append(int(board.has_castling_rights(chess.WHITE)))
                white = False
            else:
                moves_log_dict['turn'].append('black')
                moves_log_dict['Castling_right'].append(int(board.has_castling_rights(chess.BLACK)))
                white = True
                
            #(Pseudo) en passant opportunity
            moves_log_dict['EP_option'].append(int(board.has_legal_en_passant()))
            moves_log_dict['Pseudo_EP_option'].append(int(board.has_pseudo_legal_en_passant()))
            
            #Halfmove clock
            moves_log_dict['Halfmove_clock'].append(board.halfmove_clock)
                
        game_counter += 1
        if game_counter == 50:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

50 games read.
CPU times: user 765 ms, sys: 16.2 ms, total: 782 ms
Wall time: 781 ms


## Players

In [5]:
df_players = pd.DataFrame(players)
#TODO player ID
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [6]:
black = list(df_players["Black"]) 
white = list(df_players["White"])
    
#merge uniqe values from both columns:
bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])

In [7]:
def finding_comp():
    comp1 = df_players.loc[df_players['WhiteIsComp'] == "Yes"]
    comp1 = comp1[['White']].copy()
    comp1.drop_duplicates(inplace=True)
    comp2 = df_players.loc[df_players['WhiteIsComp'] == "No"]
    comp2 = comp2[['Black']].copy()
    comp2.drop_duplicates(inplace=True)
    
    c1 = list(comp1['White'])
    c2 = list(comp2['Black'])
    computer = list(set(c1 + c2))
    
    computer = pd.DataFrame({'Computer' : computer})
    computer['Yes'] = 'Yes'
    
    
    return computer

In [8]:
finding_comp()

,Computer,Yes
0,IFDThor,Yes
1,GriffySr,Yes
2,scalaQueen,Yes
3,exeComp,Yes
4,nakshatra,Yes
5,forlat,Yes
6,chesspickle,Yes
7,foggydew,Yes
8,Notarious,Yes


In [9]:
#generates unique IDs from int
def id_generator(id):
    return uuid.uuid4().int

In [10]:
#generates unique IDs containing digits and characters
def short_id_gen(id):
    return shortuuid.ShortUUID().random(length=15)

In [11]:
def game_id(input_df):
    '''generates IDs for df_games'''
    input_df['Game_ID'] = input_df['Game_ID'].apply(short_id_gen)
    df_games = input_df
    return df_games

In [12]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids
    
    computer = finding_comp()
    players_id = players_id.merge(computer, left_on='Players', right_on='Computer', how='outer')
    players_id = players_id.replace(np.nan, 'No')
    players_id.drop(columns='Computer', inplace=True)
    players_id.rename(columns={'Yes' : 'Computer'}, inplace=True)
    
    return players_id

In [13]:
players_id = players_id_list(df_players, players_id)
players_id

,Player_ID,Players,Computer
0,268715629036954651341278341546054606890,scalaQueen,Yes
1,105796795097757617856135526385647061912,Bjboy,No
2,127800785002003763210618009810584245480,Genobear,No
3,16887539725025642039443972624875385182,Qiyas,No
4,39627001233364806087170382057345563780,chesspickle,Yes
5,281155479787018396207989531982118408673,Notarious,Yes
6,214361965354043111037278401272026426328,JMM,No
7,123445714837429497660988265504044307227,GriffySr,Yes
8,231775251354672903131376427162422509122,exeComp,Yes
9,323225179299118795954881779880867980024,chenqisheng,No


In [14]:
# players_id = players_id.merge(computer, left_on='Players', right_on='Computer', how='outer')
# players_id

In [15]:
# players_id.drop(columns='Computer', inplace=True)

In [16]:
# players_id.rename(columns={'Yes': 'Computer'})
# players_id.replace(np.nan, 'No')

In [17]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [18]:
players_id.dtypes == object

Player_ID    True
Players      True
Computer     True
dtype: bool

In [ ]:
players_id.shape[0] == len(players_id["Player_ID"].unique())

In [ ]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players']).astype(object)
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
    df_players = m_bw
    return df_players

In [ ]:
assign_player_id(df_players)

In [ ]:
# def assign_player(input_df): # returns a df with 2 new columns and assigned player ID
#     #print("List od IDs has been generated:")
#     #print(players_unique)
#     m_white = pd.concat(input_df, players_id, join = 'outer')   #
#     m_white['White_ID'] = m_white['Player_ID']
#     m_white.drop(columns=['Players', "Player_ID"], inplace=True)
#     m_bw = m_white.concat(players_id, left_on=["Black"], right_on=['Players'])
#     m_bw['Black_ID'] = m_bw['Player_ID']
#     m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
#     df_players = m_bw
#     return df_players

In [ ]:
#set.union(set(df_players['White']),(set(df_players['Black'])))

## Games

In [ ]:
# def player_ids_to_games(df_games):   
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
#     return df_games

In [ ]:
df_games = pd.DataFrame(games)
#TODO date to datetime

# def game_id(input_df):
#     '''generates IDs for df_games'''
#     input_df["old_ID"] = input_df["Game_ID"]
#     input_df['Game_ID'] = input_df['Game_ID'].apply(short_id_gen)
#     return df_games

def game_id():
    df_games["old_ID"] = df_games["Game_ID"]
    df_games["Game_ID"] = df_games['Game_ID'].apply(short_id_gen)
    
    return df_games




In [ ]:
game_id()

In [ ]:
#     data = [df_games["Game_ID"], df_games["old_ID"]]
#     headers = ["Game_ID", "old_ID"]
#     df3 = pd.concat(data, axis=1, keys=headers)
#     return df3

# def game_ids(df_games):
#     old_and_new_ids()
    
#     # merge df_games with players_id
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
    
#     return df_games

# def merging_b_w_ids(df_games):
#     '''returns a df_games with additional White_ID, Black_ID'''
#     df_games["old_ID"] = df_games["Game_ID"]
#     df_games["Game_ID"] = game_id(df_games)
    
#     df_games = df_games.merge(players_id, left_on='White', right_on='Players')
#     df_games = df_games.merge(players_id, left_on='Black', right_on='Players')
#     df_games.drop(columns=['Players_x', 'Players_y'], inplace=True) #optionally drop White, Black columns
#     df_games.rename(columns = {'Game_ID_y' : 'Game_ID', 'Player_ID_x': 'White_ID', 'Player_ID_y': 'Black_ID'}, inplace=True)
#     return df_games

In [ ]:
games_dict = df_games.to_dict()

In [ ]:
'Game_ID' and 'old_ID' in games_dict.keys()

In [ ]:
games_dict.keys()

In [ ]:
df_games

## Moves

In [ ]:
df_moves = pd.DataFrame(moves_log_dict)

In [ ]:
def move_id(df_moves, df_games):  
    data = [df_games["Game_ID"], df_games["old_ID"]]
    headers = ["Game_ID", "old_ID"]
    df = pd.concat(data, axis=1, keys=headers)
    merging = df_moves.merge(df, how="left", left_on="Game_ID", right_on="old_ID")
    merging.drop(columns=["Game_ID_x", "old_ID"], inplace=True)
    merging.rename(columns = {'Game_ID_y' : 'Game_ID'}, inplace = True)
    merging.insert(0, "Moves_ID", merging.apply(lambda row: f"{row.Game_ID }-{row.turn}-{row.Halfmove_clock}", axis=1))
    df_moves = merging
    return df_moves

In [ ]:
move_id(df_moves, df_games)

In [ ]:
pgn2 = open("../raw_data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player

game_string_list = []
game_counter = 0

while True: 
    
    game = chess.pgn.read_game(pgn2)
    game_string = str(game)
    game_string_list.append(game_string)
    
    game_counter += 1
    if game_counter == 50:  # number of games to read        
        break

print(game_string_list[15])

In [ ]:
game.headers["Time"]

# search

In [ ]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')

In [ ]:
search_df(df_moves, 'ECO', 'A01')

In [ ]:
move1 = chess.Move.from_uci(uci="g8f6")
move1

In [ ]:
# move = df_moves["FEN_moves"]
# board = chess.Board(move)
# board

In [ ]:
#board.lan(move=move1)

In [ ]:
positions = board.piece_map()

In [ ]:
PIECES = list(set((val) for val in positions.values()))
PIECES

In [ ]:
SQUARES = [i for i in range(64)]

In [ ]:
for square in SQUARES:
    print(square)

In [ ]:
54 in positions.keys()

In [ ]:
bitmap_board_dict = {}

for piece in PIECES:
    bitmap_board = {}
    for square in SQUARES:
        if square in positions.keys():
            if positions[square] == piece: 
                bitmap_board[square] = 1
            else:
                bitmap_board[square] = 0
        else:
            bitmap_board[square] = 0
    bitmap_board_dict[str(piece)] = bitmap_board

In [ ]:
bitmap_board_dict

In [ ]:
board.has_legal_en_passant()

In [ ]:
board.has_pseudo_legal_en_passant()

In [ ]:
board.halfmove_clock

# Data import with package

In [ ]:
player_df2, game_df2, move_df2 = data.ChessData().import_data(data_path='../raw_data/Fics_data_pc_data.pgn', 
                                                              import_lim=50)


In [ ]:
move_df2

## Binary vector representation for board

In [ ]:
df_test = pd.DataFrame(move_df2['Bitmap_moves'][0])

In [ ]:
df_test

In [ ]:
dict_wide = {}

for index1, i in enumerate(df_test.columns):
    for index2, j in enumerate(df_test.index):
        dict_wide[str(i)+str(j)] = [df_test.iloc[index2, index1]]

In [ ]:
dict_wide

In [ ]:
pd.DataFrame(dict_wide)

In [ ]:
from cc_detector.move import binary_board_df

In [ ]:
df_wide = binary_board_df(move_df2)
#very inefficient so far (takes very long, probably needs to be redone differently)

In [ ]:
df_wide